In [1]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install streamlit

  Using cached streamlit-1.45.0-py3-none-any.whl.metadata (8.9 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
Using cached streamlit-1.45.0-py3-none-any.whl (9.9 MB)
Using cached altair-5.5.0-py3-none-any.whl (731 kB)
Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%%writefile app.py
import streamlit as st
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import time
import gzip
import os
import torch

model_name ='nq-distilbert-base-v1'
model = SentenceTransformer(model_name)
#this is the model used for generating vectors (encoding)(biencoder)

top_k = 5
#the no. of passages we want

text = r'C:\Users\avike\OneDrive\Desktop\semantic\simplewiki-2020-11-01.jsonl'
# this is the corpus data which will be put into the biencoder to generate the corpus embeddings.

corpus = []
with open(text, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data = json.loads(line.strip())
        for paragraph in data['paragraphs']:
            # We encode the passages as [title, text]
            corpus.append([data['title'], paragraph])
print("corpus:", len(corpus))


if model_name == 'nq-distilbert-base-v1':
    embeddings_filepath = 'simplewiki-2020-11-01-nq-distilbert-base-v1.pt'
    if not os.path.exists(embeddings_filepath):
        util.http_get('http://sbert.net/datasets/simplewiki-2020-11-01-nq-distilbert-base-v1.pt', embeddings_filepath)

    corpus_embeddings = torch.load(embeddings_filepath, map_location=torch.device('cpu'))
    corpus_embeddings = corpus_embeddings.float()  # Convert embedding file to float
    if torch.cuda.is_available():
        corpus_embeddings = corpus_embeddings.to('cuda')
print("Corpus embeddings size:", len(corpus_embeddings))


#now we ask for the query and embed the query.
def search(query):
    # Encode the query using the bi-encoder and find potentially relevant passages
    start_time = time.time()
    query_embedding = model.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query
    end_time = time.time()

    sorted_hits = sorted(hits, key=lambda x: x['score'], reverse=True)

    # Output of top-k hits
    results = []
    for hit in sorted_hits:
      title, paragraph = corpus[hit['corpus_id']]
      score = hit['score']
      results.append((title, paragraph, score))
    search_time = end_time - start_time
    return results, search_time
    print("Results (after {:.3f} seconds):".format(end_time - start_time))


#streamlit ui
import streamlit as st
st.title('Semantic Search Engine')
query = st.text_input('Enter your query:')
if query:
    results, search_time = search(query)
    st.write(f"Results (after {search_time:.3f} seconds):")
    for title, paragraph, score in results:
        st.write(f"**Title:** {title}")
        st.write(f"**Paragraph:** {paragraph}")
        st.write(f"**Score:** {score:.3f}")
        st.write("\n")

Overwriting app.py


In [3]:
# !streamlit run app.py